# Лабораторная работа №4. Условная оптимизация. Штрафные и барьерные функции

Цель работы: изучить подходы к решению задач оптимизации с ограничениями на примере методов штрафных и барьерных функций.

In [1]:
import numpy as np
from scipy.optimize import minimize

In [2]:
def f(x):
  return np.exp(x[0]) + np.exp(x[1])

In [71]:
def g(x):
  return -x[0] - x[1] + 1, -x[0], x[1] - 5

In [4]:
def h(x):
  return x[0]**2 + x[1]**2 - 9

### Метод штрафных функций

In [72]:
print([t for t in g((2,1.5))])
print([max(0, t) for t in g((2,1.5))])

[-2.5, -2, -3.5]
[0, 0, 0]


In [6]:
h((2,1.5)), np.abs(h((2,1.5)))

(-2.75, 2.75)

In [16]:
#штрафная функция
def a(x, p=1):
  return np.sum(np.array([max(0, y) for y in g(x)])**p) + np.abs(h(x))**p

In [129]:
def penalty(x0, mu, b, eps=0.01):
  xk = x0
  iters = 0
  while True:
    iters += 1
    result = minimize(lambda x, r: f(x)+r*a(x), xk, args=(mu,))
    xk = result.x
    if mu*a(xk) < eps: return iters, result
    mu *= b

In [130]:
iters, result = penalty((2, 1.5), 1, 10)
print("count iters", iters)
print("x* =", result.x)
print("f* =", result.fun)

count iters 2
x* = [2.78158519 1.12373654]
f* = 19.22092064769987


### Метод барьерных функций

In [126]:
def B(x, method):
  if method == 'inverse':
    return np.sum(-1 / np.array(g(x)))
  if method == 'logarithmic':
    return np.sum(-np.log(g(x)))

In [127]:
def barrier(x0, mu, b, method='inverse', eps=0.01):
  xk = x0
  iters = 0
  while True:
    iters += 1
    result = minimize(lambda x, r: f(x)+r*B(x, method), xk, args=(mu,),
                      constraints=[{'type': 'ineq', 'fun': lambda x: -x[0] - x[1] + 1 - eps},
                                   {'type': 'ineq', 'fun': lambda x: -x[0] - eps},
                                   {'type': 'ineq', 'fun': lambda x: -x[0] - x[1] + 1 - eps}])
    xk = result.x
    if abs(mu*B(xk, method)) < eps: return iters, result
    mu *= b

In [128]:
iters, result = barrier((2, 1.5), 10, 0.5, method='inverse')
print("count iters", iters)
print("x* =", result.x)
print("f* =", result.fun)

count iters 18
x* = [-13.44964853 -12.12330334]
f* = 2.785101078890199e-06


<ipython-input-2-85b10a2b45ff>:2: RuntimeWarning: overflow encountered in exp
  return np.exp(x[0]) + np.exp(x[1])


In [120]:
iters, result = barrier((2, 1.5), 10, 0.4, method='logarithmic')
print("count iters", iters)
print("x* =", result.x)
print("f* =", result.fun)

<ipython-input-77-9e1463920a03>:5: RuntimeWarning: invalid value encountered in log
  return -np.sum(np.log(g(x)))


KeyboardInterrupt: ignored